# Lean 6 - Mathlib4 : La Bibliotheque Mathematique

## Introduction

Ce notebook explore **Mathlib4**, la bibliotheque mathematique communautaire pour Lean 4. Avec plus de **4 millions de lignes de code** formalise, **~8000 modules**, et plus de **100 contributeurs reguliers**, Mathlib est devenue la reference mondiale pour les mathematiques formelles.

### Pourquoi Mathlib est revolutionnaire

- **Verification formelle** : Chaque theoreme est prouve par machine, garantissant l'absence d'erreurs
- **Reutilisabilite** : Les preuves existantes servent de base pour de nouvelles decouvertes
- **Collaboration** : Des mathematiciens du monde entier contribuent
- **IA-ready** : Base d'entrainement pour les systemes comme AlphaProof et LeanCopilot

### Objectifs pedagogiques

1. Comprendre la structure et l'organisation de Mathlib4
2. Installer et configurer Mathlib dans un projet Lean
3. Maitriser les tactiques puissantes : `ring`, `linarith`, `omega`, `norm_num`
4. Naviguer dans la documentation et chercher des theoremes
5. Utiliser Mathlib pour des preuves non triviales

### Prerequis

- Notebooks **Lean-1** a **Lean-5** completes
- Connexion internet pour telecharger Mathlib

### Duree estimee : 45-50 minutes

---

## Contexte : L'Ecosysteme Mathlib

### Historique

| Annee | Evenement |
|-------|-----------|
| 2017 | Creation de Mathlib pour Lean 3 |
| 2023 | Migration complete vers Lean 4 (Mathlib4) |
| 2024 | Utilisation par AlphaProof (IMO medaille argent) |
| 2025 | Terry Tao formalise des resultats en theorie analytique |
| 2026 | Lean4Lean presente a POPL (bootstrap de Lean en Lean) |

### Etat actuel (Janvier 2026)

- **Version** : 4.27.0-rc1
- **Taille** : 4+ millions de lignes, ~8000 modules
- **Contributeurs** : 100+ reguliers, 500+ occasionnels
- **Utilisateurs notables** : Terry Tao (Fields Medal), Kevin Buzzard (Imperial)
- **Repository** : https://github.com/leanprover-community/mathlib4
- **Documentation** : https://leanprover-community.github.io/mathlib4_docs/

## 1. Installation et Configuration

### 1.1 Creer un projet avec Mathlib

Mathlib s'installe via **Lake**, le gestionnaire de paquets Lean 4.

### 1.1 Creer un projet avec Mathlib (Terminal)

Mathlib s'installe via **Lake**, le gestionnaire de paquets Lean 4. Voici les commandes a executer dans un terminal :

```bash
# Creer un nouveau projet avec Mathlib
lake new mon_projet math
cd mon_projet

# Telecharger les dependances
lake update

# Telecharger le cache compile (IMPORTANT - economise des heures!)
lake exe cache get
```

**Structure du projet resultant :**

```
mon_projet/
├── lakefile.lean      # Configuration Lake
├── lean-toolchain     # Version de Lean
├── MonProjet.lean     # Point d'entree
└── lake-manifest.json # Versions des dependances
```

**Note** : Le telechargement initial de Mathlib4 peut prendre 10-20 minutes et ~1 Go d'espace disque. Le cache precompile (`lake exe cache get`) evite de tout recompiler.

### 1.2 lakefile.lean type pour Mathlib

Le fichier `lakefile.lean` configure les dependances du projet. Pour Mathlib, il specifie la version et permet de telecharger le cache precompile avec `lake exe cache get`.

In [ ]:
-- Exemple de lakefile.lean pour un projet Mathlib
/-
import Lake
open Lake DSL

package "mon_projet" where
  version := v!"0.1.0"

require mathlib from git
  "https://github.com/leanprover-community/mathlib4"

@[default_target]
lean_lib «MonProjet» where
  -- Configuration additionnelle si necessaire
-/

### 1.3 Importer Mathlib

Les imports Mathlib suivent une hierarchie : `Mathlib.Data.Nat.Prime` pour les nombres premiers, `Mathlib.Algebra.Ring.Basic` pour les anneaux, etc.

In [ ]:
-- Imports typiques (a utiliser dans un projet avec Mathlib)
-- import Mathlib.Tactic              -- Toutes les tactiques
-- import Mathlib.Data.Nat.Prime      -- Nombres premiers
-- import Mathlib.Algebra.Ring.Basic  -- Structures d'anneaux
-- import Mathlib.Analysis.SpecialFunctions.Log.Basic  -- Logarithmes

-- Pour ce notebook sans Mathlib installe, on simule les concepts
-- Les exemples sont documentes mais peuvent necessiter Mathlib pour s'executer

## 2. Structure de Mathlib4

### 2.1 Organisation des modules

Mathlib est organise en namespaces thematiques :

| Namespace | Contenu | Exemples |
|-----------|---------|----------|
| `Mathlib.Data` | Structures de donnees | List, Set, Finset, Multiset |
| `Mathlib.Algebra` | Structures algebriques | Group, Ring, Field, Module |
| `Mathlib.Order` | Relations d'ordre | Lattice, Complete lattice |
| `Mathlib.Topology` | Topologie generale | Espaces topologiques, compacite |
| `Mathlib.Analysis` | Analyse reelle/complexe | Limites, derivees, integrales |
| `Mathlib.NumberTheory` | Theorie des nombres | Primalite, congruences, Legendre |
| `Mathlib.Logic` | Logique avancee | Logique classique, cardinalite |
| `Mathlib.Tactic` | Tactiques Mathlib | ring, linarith, omega, polyrith |

### 2.2 Hierarchie de structures algebriques

Mathlib definit une hierarchie riche de structures algebriques :

```
Semigroup -> Monoid -> Group
    |           |         |
    v           v         v
CommSemigroup -> CommMonoid -> CommGroup
    |           |         |
    v           v         v
Semiring  ->   Ring   ->  Field
```

Cette hierarchie utilise les **typeclasses** de Lean pour l'inference automatique.

In [ ]:
-- Exemple conceptuel de typeclass (syntaxe simplifiee)
-- class Monoid (M : Type*) extends One M, Mul M where
--   one_mul : forall a : M, 1 * a = a
--   mul_one : forall a : M, a * 1 = a
--   mul_assoc : forall a b c : M, (a * b) * c = a * (b * c)

-- Nat est automatiquement un Monoid additif et multiplicatif
#check @Nat.add_comm      -- Commutativity de l'addition
#check @Nat.mul_assoc     -- Associativite de la multiplication

## 3. Tactiques Mathlib Essentielles

Mathlib fournit des tactiques **puissantes** qui automatisent de nombreuses preuves. Ces tactiques ne sont pas dans Lean de base mais deviennent indispensables pour les mathematiques serieuses.

### 3.1 `ring` : Algebre polynomiale

La tactique `ring` resout automatiquement les egalites dans les anneaux (commutatifs ou non).

In [ ]:
-- Exemples avec ring (necessite Mathlib.Tactic)
-- Ces exemples fonctionnent dans un projet avec Mathlib installe

-- Identite remarquable
-- example (a b : Int) : (a + b)^2 = a^2 + 2*a*b + b^2 := by ring

-- Factorisation
-- example (a b : Int) : a^2 - b^2 = (a + b) * (a - b) := by ring

-- Expression complexe
-- example (x y z : Int) : 
--   (x + y + z)^2 = x^2 + y^2 + z^2 + 2*x*y + 2*x*z + 2*y*z := by ring

-- Pour Nat (sans Mathlib), on peut utiliser des lemmes manuellement
-- Voici une version simplifiee demontrant le concept
theorem double_times (a : Nat) : a + a = 2 * a := by
  simp [Nat.two_mul]

-- Avec Mathlib installe, on prouverait directement :
-- theorem square_expand (a b : Nat) : (a + b) * (a + b) = a*a + 2*a*b + b*b := by ring

### 3.2 `linarith` : Arithmetique lineaire

La tactique `linarith` resout automatiquement les inegalites lineaires sur les nombres reels, rationnels ou entiers. Elle combine les hypotheses pour trouver une contradiction.

In [ ]:
-- Exemples avec linarith (Mathlib.Tactic)

-- Inegalite simple
-- example (x : Int) (h : x > 5) : x >= 5 := by linarith

-- Combinaison d'hypotheses
-- example (x y : Int) (h1 : x <= y) (h2 : y <= x + 3) : y - x <= 3 := by linarith

-- Systeme d'inegalites
-- example (a b c : Int) (h1 : a + b > c) (h2 : b + c > a) (h3 : c + a > b) :
--   a + b + c > 0 := by linarith

-- Version avec omega (disponible dans Lean de base pour Nat/Int)
theorem ineq_example (x y : Nat) (h1 : x <= y) (h2 : y <= x + 3) : y - x <= 3 := by
  omega

### 3.3 `omega` : Arithmetique Presburger

La tactique `omega` est un decideur complet pour l'arithmetique de Presburger (entiers avec +, -, comparaisons). Plus puissante que `linarith` pour les entiers.

In [ ]:
-- omega est disponible dans Lean de base pour Nat et Int

-- Egalites et inegalites
theorem omega_demo1 (n m : Nat) : n + m = m + n := by omega

theorem omega_demo2 (n : Nat) (h : n > 0) : n >= 1 := by omega

-- Systemes complexes
theorem omega_complex (a b c : Int)
  (h1 : a + b = c) (h2 : a - b = 0) : 2 * a = c := by omega

-- Divisibilite (modulo)
-- theorem div_mod (n : Nat) : n = n / 2 * 2 + n % 2 := by omega

### 3.4 `norm_num` : Calcul numerique

La tactique `norm_num` evalue les expressions numeriques et verifie les proprietes calculables (primalite, divisibilite, etc.). Indispensable pour les assertions concretes.

In [ ]:
-- Exemples avec norm_num (Mathlib.Tactic.NormNum)

-- Calculs simples
-- example : (2 : Nat) + 2 = 4 := by norm_num
-- example : (123 : Nat) * 456 = 56088 := by norm_num

-- Primalite
-- example : Nat.Prime 17 := by norm_num
-- example : ¬ Nat.Prime 15 := by norm_num

-- Comparaisons
-- example : (100 : Nat) < 200 := by norm_num

-- Sans Mathlib, on utilise decide ou rfl
theorem calc_example : 2 + 2 = 4 := by rfl
theorem compare_example : 5 < 10 := by decide

### 3.5 `field_simp` : Simplification dans les corps

`field_simp` simplifie les expressions avec divisions et fractions.

In [ ]:
-- Exemples avec field_simp (Mathlib.Tactic.FieldSimp)

-- Fractions
-- example (a b : Rat) (hb : b ≠ 0) : a / b * b = a := by field_simp

-- Addition de fractions
-- example (a b c : Rat) (hb : b ≠ 0) (hc : c ≠ 0) :
--   a / b + a / c = a * (b + c) / (b * c) := by field_simp; ring

-- Simplification complexe
-- example (x : Real) (hx : x ≠ 0) : (x + 1) / x - 1 / x = 1 := by
--   field_simp
--   ring

### 3.6 `polyrith` : Certificats polynomiaux

`polyrith` trouve automatiquement des preuves polynomiales en interrogeant **Sage** (systeme de calcul formel).

In [ ]:
-- Exemples avec polyrith (Mathlib.Tactic.Polyrith)
-- Necessite une connexion a un serveur Sage

-- Egalite polynomiale non triviale
-- example (a b : Int) (h1 : a + b = 4) (h2 : a - b = 2) : a = 3 := by polyrith

-- Systeme d'equations
-- example (x y : Rat) (h1 : 2*x + 3*y = 7) (h2 : x - y = 1) : 
--   x = 2 := by polyrith

## 4. Exemples de Theoremes Mathlib

### 4.1 Theorie des nombres

In [ ]:
-- Avec import Mathlib.Data.Nat.Prime

-- Primalite
-- #check Nat.Prime                    -- Definition de nombre premier
-- #check Nat.Prime.two_le             -- p premier => p >= 2
-- #check Nat.Prime.one_lt             -- p premier => 1 < p
-- #check Nat.minFac_prime            -- Plus petit facteur premier

-- Exemple : prouver que 17 est premier
-- theorem prime_17 : Nat.Prime 17 := by
--   norm_num

-- Theoreme d'Euclide (infinite de premiers) existe dans Mathlib
-- #check Nat.exists_infinite_primes

### 4.2 Algebre

In [ ]:
-- Avec import Mathlib.Algebra.Group.Basic

-- Proprietes des groupes
-- #check mul_one                -- a * 1 = a
-- #check one_mul                -- 1 * a = a
-- #check mul_assoc              -- (a * b) * c = a * (b * c)
-- #check mul_inv_cancel         -- a * a⁻¹ = 1

-- Proprietes des anneaux
-- #check add_mul                -- (a + b) * c = a * c + b * c
-- #check mul_add                -- a * (b + c) = a * b + a * c
-- #check neg_add_cancel         -- -a + a = 0

-- Ces theoremes s'appliquent a TOUS les types avec la structure adequate
-- (Int, Rat, Real, Complex, matrices, polynomes, etc.)

### 4.3 Analyse

In [ ]:
-- Avec import Mathlib.Analysis.Calculus.Deriv.Basic

-- Derivees
-- #check HasDerivAt            -- Definition de la derivee
-- #check deriv_add             -- (f + g)' = f' + g'
-- #check deriv_mul             -- (f * g)' = f' * g + f * g'
-- #check deriv_pow             -- (x^n)' = n * x^(n-1)

-- Limites
-- #check Filter.Tendsto        -- Definition des limites
-- #check tendsto_add           -- lim(f + g) = lim f + lim g
-- #check tendsto_mul           -- lim(f * g) = lim f * lim g

-- Integration
-- #check MeasureTheory.integral_add  -- Linearite de l'integrale

## 5. Recherche et Navigation

### 5.1 Documentation en ligne

La documentation Mathlib est disponible sur : https://leanprover-community.github.io/mathlib4_docs/

### 5.2 Loogle : Recherche syntaxique

**Loogle** (https://loogle.lean-lang.org/) permet de chercher des theoremes par leur signature de type.

In [ ]:
-- Exemples de requetes Loogle :

-- Chercher : "theoreme sur addition et zero"
-- Requete : ?a + 0 = ?a
-- Resultat : Nat.add_zero, Int.add_zero, etc.

-- Chercher : "commutativite de l'addition"
-- Requete : ?a + ?b = ?b + ?a

-- Chercher : "tout theoreme impliquant Prime et Odd"
-- Requete : Nat.Prime, Nat.Odd

-- Syntaxe avancee :
-- ?a     : variable de type quelconque
-- _      : argument dont on ne se soucie pas
-- A -> B : fonction de A vers B

### 5.3 Moogle : Recherche semantique

**Moogle** (https://www.moogle.ai/) utilise des LLMs pour la recherche semantique dans Mathlib - on peut chercher en langage naturel.

In [ ]:
-- Exemples de requetes Moogle (langage naturel) :

-- "derivative of exponential function"
-- -> Real.deriv_exp : deriv exp x = exp x

-- "infinitely many prime numbers"
-- -> Nat.exists_infinite_primes

-- "triangle inequality"
-- -> norm_add_le : ||a + b|| <= ||a|| + ||b||

-- "fundamental theorem of calculus"
-- -> intervalIntegral.integral_eq_sub_of_hasDerivAt

### 5.4 Commandes de recherche dans Lean

In [ ]:
-- exact? cherche une preuve exacte
-- example (n : Nat) : n + 0 = n := by exact?
-- Resultat : exact Nat.add_zero n

-- apply? cherche un lemme a appliquer
-- example (n m : Nat) : n <= n + m := by apply?
-- Resultat : apply Nat.le_add_right

-- rw? suggere des reecritures
-- example (n : Nat) : 0 + n = n := by rw?
-- Resultat : rw [Nat.zero_add]

## 6. Cas d'Usage Avances

### 6.1 Terry Tao et la theorie analytique des nombres

En janvier 2026, **Terence Tao** (medaille Fields 2006) a utilise Mathlib4 pour formaliser des resultats en theorie analytique des nombres. Il a notamment contribue a :

- Formalisation de lemmes sur les fonctions L de Dirichlet
- Verification de bornes asymptotiques pour la fonction pi(x)
- Travail vers une preuve formelle complete du theoreme des nombres premiers

Cet evenement marque un tournant : un mathematicien de premier plan utilise la verification formelle comme outil de recherche quotidien.

### 6.2 Theoremes majeurs formalises dans Mathlib

| Theoreme | Domaine | Auteur original |
|----------|---------|-----------------|
| Dernier theoreme de Fermat | Theorie des nombres | Fermat/Wiles |
| Theoreme des nombres premiers | Analyse | Hadamard/de la Vallee Poussin |
| Theoreme de Bolzano-Weierstrass | Analyse | Bolzano |
| Lemme de Zorn | Theorie des ensembles | Zorn |
| Theoreme de Hahn-Banach | Analyse fonctionnelle | Hahn/Banach |
| Theoreme de Stone-Weierstrass | Approximation | Stone/Weierstrass |

### 6.3 Lean4Lean : Le bootstrap (POPL 2026)

**Lean4Lean** est un projet ambitieux qui vise a reimplementer Lean 4 en Lean 4 lui-meme. Presente a POPL 2026, il demontre :

- **Auto-hebergement** : Lean peut se compiler lui-meme
- **Verification** : Le compilateur est prouve correct
- **Confiance accrue** : Reduit la base de code "de confiance"

### 6.4 Impact sur la recherche mathematique

**Avant la formalisation** :
- Revue par pairs humaine, erreurs possibles
- Temps de verification : mois/annees
- Confiance : depend de la reputation

**Avec Mathlib/Lean** :
- Verification automatique, certitude mathematique
- Temps : minutes/heures pour verifier
- Confiance : absolue si la preuve compile

## 7. Exercices

### Exercice 1 : Utiliser omega

In [ ]:
-- Prouver avec omega
theorem ex1 (a b c : Nat) (h1 : a + b = c) (h2 : c < 10) : a < 10 := by
  sorry

### Exercice 2 : Calculer avec rfl/decide

In [ ]:
-- Prouver ces calculs
theorem ex2a : 7 * 8 = 56 := by sorry
theorem ex2b : 100 > 50 := by sorry

### Exercice 3 : Preuve manuelle style Mathlib

In [ ]:
-- Prouver l'associativite de l'addition sur Nat
-- (sans utiliser directement Nat.add_assoc)
theorem ex3 (a b c : Nat) : (a + b) + c = a + (b + c) := by
  sorry

## Solutions

In [ ]:
-- Solution exercice 1
theorem ex1_sol (a b c : Nat) (h1 : a + b = c) (h2 : c < 10) : a < 10 := by
  omega

-- Solution exercice 2
theorem ex2a_sol : 7 * 8 = 56 := by rfl
theorem ex2b_sol : 100 > 50 := by decide

-- Solution exercice 3
-- On utilise directement le lemme de la bibliotheque standard
theorem ex3_sol (a b c : Nat) : (a + b) + c = a + (b + c) :=
  Nat.add_assoc a b c

## Resume

### Tactiques Mathlib essentielles

| Tactique | Domaine | Usage | Disponibilite |
|----------|---------|-------|---------------|
| `ring` | Algebre | Egalites polynomiales | Mathlib |
| `linarith` | Arithmetique | Inegalites lineaires | Mathlib |
| `omega` | Presburger | Arithmetique avec +, -, < | Lean de base |
| `norm_num` | Calcul | Expressions numeriques | Mathlib |
| `field_simp` | Corps | Fractions et divisions | Mathlib |
| `polyrith` | Polynomes | Certificats via Sage | Mathlib |
| `simp` | General | Simplification automatique | Lean de base |
| `exact?` | Recherche | Trouver une preuve exacte | Mathlib |
| `apply?` | Recherche | Trouver un lemme applicable | Mathlib |
| `decide` | Decision | Propositions decidables | Lean de base |
| `rfl` | Egalite | Reflexivite definitionnelle | Lean de base |

### Structure Mathlib4

| Namespace | Contenu principal |
|-----------|-------------------|
| `Mathlib.Data` | List, Set, Finset, Multiset, Nat, Int, Rat |
| `Mathlib.Algebra` | Group, Ring, Field, Module, LinearMap |
| `Mathlib.Order` | Lattice, Order, Filter |
| `Mathlib.Topology` | TopologicalSpace, Compactness, Continuity |
| `Mathlib.Analysis` | Limits, Derivatives, Integrals, Measures |
| `Mathlib.NumberTheory` | Prime, Divisibility, Modular arithmetic |
| `Mathlib.Tactic` | ring, linarith, omega, norm_num, etc. |

### Ressources

| Ressource | URL | Usage |
|-----------|-----|-------|
| Documentation officielle | https://leanprover-community.github.io/mathlib4_docs/ | Reference complete |
| Loogle | https://loogle.lean-lang.org/ | Recherche par signature de type |
| Moogle | https://www.moogle.ai/ | Recherche en langage naturel |
| Repository GitHub | https://github.com/leanprover-community/mathlib4 | Code source |
| Zulip Chat | https://leanprover.zulipchat.com/ | Communaute et support |
| Mathematics in Lean | https://leanprover-community.github.io/mathematics_in_lean/ | Tutoriel approfondi |

### Commandes de recherche

```lean
-- exact? : trouve une preuve exacte
-- apply? : trouve un lemme a appliquer  
-- rw?    : suggere des reecritures
-- simp?  : montre les regles simp utilisees
```

### Prochaine etape

Dans le notebook **Lean-7-LLM-Integration**, nous verrons comment les **Large Language Models** peuvent assister la construction de preuves Lean, avec des outils comme **LeanCopilot** (NeurIPS 2025), **LeanProgress** (TMLR 2025), et les percees recentes d'**AlphaProof** (Nature 2025).

---

*Notebook base sur l'etat de l'art Mathlib4 v4.27.0-rc1 (janvier 2026)*
*Ressources : leanprover-community.github.io, Terry Tao blog, Lean4Lean (POPL 2026)*